<a href="https://colab.research.google.com/github/gasoaress/BattleShip/blob/main/PavementsCrash-VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Preparação dos dados
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory='/content/Images/Train', target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory='/content/Images/Test', target_size=(224,224))

# Modelo VGG16
VGG = keras.applications.VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
VGG.trainable = False

model = Sequential([
    VGG,
    Flatten(),
    Dense(units=256, activation="relu"),
    Dense(units=256, activation="relu"),
    Dense(units=2, activation="softmax")
])

model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

model.summary()

# Treinamento do modelo
hist = model.fit(
    traindata,
    steps_per_epoch=100,
    validation_data=testdata,
    validation_steps=10,
    epochs=5
)

model.save('vggclf.h5')

# Verificar as chaves no histórico do treinamento
print(hist.history.keys())

# Visualização do treinamento
import matplotlib.pyplot as plt

plt.plot(hist.history["accuracy"])
if "val_accuracy" in hist.history:
    plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history['loss'])
if "val_loss" in hist.history:
    plt.plot(hist.history['val_loss'])

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
legend_labels = ["Accuracy", "Loss"]
if "val_accuracy" in hist.history:
    legend_labels.append("Validation Accuracy")
if "val_loss" in hist.history:
    legend_labels.append("Validation Loss")
plt.legend(legend_labels)
plt.show()

# Predição
from keras.preprocessing import image
img = image.load_img("/content/paredeLisa.jpg", target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)

from keras.models import load_model
saved_model = load_model("vggclf.h5")
output = saved_model.predict(img)
if output[0][0] > output[0][1]:
    print("Sem Rachado")
else:
    print("Rachado")


Found 1078 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_15 (Dense)            (None, 256)               6422784   
                                                                 
 dense_16 (Dense)            (None, 256)               65792     
                                                                 
 dense_17 (Dense)            (None, 2)                 514       
                                                                 
Total params: 21203778 (80.89 MB)
Trainable params: 6489090 (24.75 MB)
Non-trainable params: 14714688 (56